# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [39]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [40]:
from crewai import Agent, Task, Crew

In [41]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["OPENAI_API_KEY"] = 'sk-'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner


In [42]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [43]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [44]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

### Task: Plan

In [45]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    output_file="./output/plan.md",
)

### Task: Write

In [46]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    output_file="./output/write.md",
)

### Task: Edit

In [47]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    output_file="./output/edit.md",
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [48]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-24 10:31:08,736 - 140704717815744 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [49]:
result = crew.kickoff(inputs={"topic":"Beach Yoga for new students."})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Beach Yoga for new students..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Ultimate Guide to Beach Yoga for New Students

Introduction:
- Brief overview of the benefits of practicing yoga on the beach
- Importance of starting a yoga practice as a new student 
- Mention of key players in the beach yoga industry

Key Points:
1. Latest Trends in Beach Yoga for New Students
- Rise in popularity of beach yoga retreats and classes
- Incorporating elements of nature into yoga practice
- Benefits of practicing yoga on the beach for mental and physical health

2. Key Players i

- Display the results of your execution as markdown in the notebook.

In [50]:
from IPython.display import Markdown
Markdown(result)

# The Ultimate Guide to Beach Yoga for New Students

## Introduction
Are you a new student looking to explore the world of yoga in a unique and serene environment? Look no further than beach yoga! Practicing yoga on the beach offers a multitude of benefits for both the mind and body, making it an ideal choice for beginners. Not only does it provide an opportunity to connect with nature, but it also allows for a sense of calm and relaxation that is hard to find in a traditional studio setting. In this ultimate guide, we will delve into the world of beach yoga for new students, exploring the latest trends, key players in the industry, and noteworthy news to help you prepare for your first beach yoga experience.

## Latest Trends in Beach Yoga for New Students
In recent years, beach yoga has experienced a surge in popularity, with more and more individuals opting to attend beach yoga retreats and classes. The allure of practicing yoga on the beach lies in the ability to incorporate elements of nature into your practice, from the sound of crashing waves to the feel of sand beneath your feet. This unique environment not only enhances the physical benefits of yoga but also promotes mental well-being. The combination of movement, breath, and the great outdoors creates a holistic experience that can leave new students feeling rejuvenated and at peace.

## Key Players in Beach Yoga for New Students
As you embark on your beach yoga journey, you may come across well-known yoga instructors who offer beach yoga classes in popular destinations around the world. These instructors are experienced in guiding new students through their practice, ensuring a safe and enjoyable experience by the sea. Additionally, you may encounter testimonials from new students who have experienced beach yoga firsthand, sharing their transformative experiences and the sense of community they have found through their practice. By learning from these key players in the beach yoga industry, new students can gain valuable insights and encouragement as they explore this unique form of yoga.

## Noteworthy News in Beach Yoga for New Students
Recent studies have highlighted the positive effects of beach yoga on stress reduction, showcasing the potential for this practice to promote mental and emotional well-being. In addition to research findings, there are a variety of events and workshops tailored specifically for new students to join and learn more about beach yoga. These opportunities provide a chance to connect with like-minded individuals, deepen your practice, and immerse yourself in the beauty of the beach environment. For new students preparing for their first beach yoga experience, there are tips available to help you feel confident and ready to embrace this transformative practice.

In conclusion, beach yoga offers a unique and refreshing approach to yoga practice for new students. By exploring the latest trends, connecting with key players in the industry, and staying informed on noteworthy news, new students can embark on their beach yoga journey with confidence and enthusiasm. Whether you are seeking stress relief, physical wellness, or a sense of community, beach yoga has something to offer for everyone. So grab your mat, head to the nearest beach, and experience the magic of beach yoga for yourself. Namaste.